# $Active=Confirmed-Deceased-Recovered$
- [x] I have used Active cases = # of active cases on last day of week(overlapping)
- [x] Here using overlapping weeks helps in smoothing data, since we have data points every 3 and 4 days, rather than every 7 days
- [x] I have used Active cases = # of active cases on last day of month(this is 14th of next month): as months are from 15th of previous month to 14th of next month

- **Note** run all cell in one go as I have used same variable names and they get updated as we go from district to state to overall, then if one runs previous code then comes error that this variable has different length etc.


#### Import modules

In [1]:
import json 
import pandas as pd
import numpy as np
import datetime
import scipy.signal as signal ## new module

#### Load Q1 json

In [2]:
# to open json file
f=open('neighbor-districts-modified.json')
# this function basically stores json type files into python dictionary
dist_modified=json.load(f)

#### Get district names and ids form Q1 json

In [3]:
district_list_from_json=[]
for key in dist_modified:
    district_list_from_json.append(key)
district_list_from_json=np.array(district_list_from_json)
district_list_from_json.sort()

state_district_codes=[]
for i in range(len(district_list_from_json)):
    state_district_codes.append(district_list_from_json[i].split('/')[1])

# district names - sample entry: churu
district_names_from_json=[] 

# district ids - sample entry: Q1090006
district_ids_from_json=[]

#use split() function and specify the separator '/' . Remember default seperator is whitspace
for i in range(len(district_list_from_json)):
    district_names_from_json.append(district_list_from_json[i].split("/")[0])
    district_ids_from_json.append(district_list_from_json[i].split("/")[1])

In [4]:
district_ids_list={}
for i in range(len(district_names_from_json)):
    district_ids_list[district_ids_from_json[i]]=district_names_from_json[i]

#### Creating time ids
- [x] Start date: 2020-03-15
- [x] End date: 2021-08-14
- [x] With overlapping of weeks we have total 148 weeks

In [5]:
# dictionaries for mapping dates to time ids
# eg. for 2020-3-15, time_id_week is 1, time_id_month is 1, time_id_ is 1

time_id_week = {}
time_id_month = {}
time_id_overall = {}

date=datetime.date(2020,3,15)
day=1
even_list=[]
odd_list=[]
for i in range(200):
    if (i+1)%2==0:
        even_list.append(i+1)
    else:
        odd_list.append(i+1)
while True:
    # basically to cover overlapping weeks this part needs to be changed.
    # for now we are proceeding. but change week ids according above definition of 7-DMA
    list_week=[0,1,2,6]
    if date.weekday() in list_week:
        time_id_week[str(date)]=odd_list[int(np.ceil(day/7))-1]
    else:
        time_id_week[str(date)]=even_list[int(np.ceil(day/7))-1]
        
    if str(date)[0:4]=='2020':
        if int(str(date)[8:10]) <15:
            time_id_month[str(date)]=int(str(date)[5:7])-3
        else:
            time_id_month[str(date)]=int(str(date)[5:7])-2
    else:
        if int(str(date)[8:10]) <15:
            time_id_month[str(date)]=int(str(date)[5:7])+9
        else:
            time_id_month[str(date)]=int(str(date)[5:7])+10

    time_id_overall[str(date)]=1
    
    if date==datetime.date(2021,8,14):
        break
    day=day+1
    date=date+datetime.timedelta(days=1)

#### Read districts csv for cases

In [6]:
data_csv=pd.read_csv('districts.csv')
data_csv=data_csv.drop('Tested',axis=1)

In [7]:
data_csv=data_csv.sort_values(['District','Date'])
data_csv.reset_index(inplace=True,drop=True)

#### Getting intersection district names from districts csv and modified json(same as Q3)

In [8]:
district_names_from_cases=[]
district_ids_from_cases=[]
district_uniques=np.array(np.unique(data_csv['District']))
for i in range(len(district_ids_from_json)):
    a=district_ids_from_json[i].split('_')
    for j in range(len(district_uniques)):
        if district_uniques[j]==a[1]:
            district_ids_from_cases.append(district_ids_from_json[i])
            district_names_from_cases.append(a[1])
            break

In [9]:
print("these many districts are there in districts.csv =",len(district_uniques)+5) # for those common five names
print("these many districts are in both files intersection =" ,len(district_ids_from_cases)) # though they are included in here

these many districts are there in districts.csv = 648
these many districts are in both files intersection = 622


In [10]:
print('-------To check whether things are consistent------')
print(district_ids_from_cases[67])
print(district_names_from_cases[67])

-------To check whether things are consistent------
MH_Beed
Beed


#### To get daily cases from cummulative confirmed cases
- [x] Formula use: $Active=Confirmed-Deceased-Recovered$


In [11]:
data_csv['Active']=data_csv['Confirmed']-(data_csv['Deceased']+data_csv['Recovered'])

#### Giving week and month ID to each date

In [12]:
dates_in_raw=np.unique(data_csv['Date']).tolist()

In [13]:
%%time
data_csv['Week ID']=np.nan
data_csv['Month ID']=np.nan
for date in time_id_week:
    if dates_in_raw.count(date)>0:
        data_csv.loc[data_csv['Date']==date,'Week ID']=time_id_week[date]
        data_csv.loc[data_csv['Date']==date,'Month ID']=time_id_month[date]

CPU times: user 16.9 s, sys: 13.7 ms, total: 16.9 s
Wall time: 17 s


#### Dropping rows with out of range dates
- [x] These many dates are after end date: 2021-08-14 (for all districts in intersection)

In [14]:
data_csv.isnull().sum()

Date             0
State            0
District         0
Confirmed        0
Recovered        0
Deceased         0
Other            0
Active           0
Week ID      12502
Month ID     12502
dtype: int64

In [15]:
data_csv.dropna(inplace=True)

## Analysis for Districts


- [x] Delhi also can be included but I dont't think I should write special code just to include one District.

#### Week timeline
- [x] I have used Active cases = # of active cases on last day of week(overlapping)
- [x] Here using overlapping weeks helps in smoothing data, since we have data points every 3 and 4 days, rather than every 7 days

In [16]:
%%time
no_of_weeks=list(time_id_week.values())[-1]
districtid=[]
weekid=[]
cases=[] #active cases
for i in range(len(district_names_from_cases)):
    data_foo=data_csv[data_csv['District']==district_names_from_cases[i]]
    for j in range(no_of_weeks):
        districtid.append(district_ids_from_cases[i])
        weekid.append(j+1)
        foo_df=data_foo[data_foo['Week ID']==j+2]
        if foo_df.shape[0]==0:
            cases.append(0)
        else:
            cases.append(foo_df.Active.tolist()[-1])
        #active.append(foo_df['Active'].sum())
        
week_df=pd.DataFrame({'districtid':districtid,'weekid':weekid,'cases':cases})

CPU times: user 40.7 s, sys: 3.85 ms, total: 40.7 s
Wall time: 40.6 s


#### Function that chooses between more than two peaks finded by $argrelextrema$
- [x] discription is given in python comments
- [x] there is one special case of MN_Churachandpur, where everything is zero, I have given it avg peak1 and peak2 values, manually.

In [17]:
def choose_function_week(time_series,indices):
    
    indices=indices.tolist()
    
    foo_list=time_series[indices]
    max1=max(foo_list.tolist()) # find first maxima
    ind1=indices[foo_list.argmax()] # find fist maxima's index
    indices.remove(ind1)
    
    time_series[ind1]=-99999
    foo_list=time_series[indices]
    max2=max(foo_list.tolist()) # find second maxima
    if max1==max2: # to cover the cases where value of  peaks is equal and that gives nearby peak1 and peak2 values
      ind_foo=indices[foo_list.argmax()]
      indices.remove(ind_foo)
      time_series[ind_foo]=-99999
      foo_list=time_series[indices]
      ind2=indices[foo_list.argmax()]
    else:
      ind2=indices[foo_list.argmax()] # same thing can be done for more than two max (but those cases are like 3-4), and rather complicates
    
    ### for cases like MN_Churachandpur
    ### where there is no record of cases
    if max1==0:# there is only one district like this MN_Churachandpur 
      ind1=50  # if remove this then this gives peaks some cakcy value
      ind2=120 # for that giving a nice value around avg of other district peaks
    
    # return index
    if ind1<ind2: # swap index when peak1_week > peak2_week
        return [ind1,ind2]
    else:
        return [ind2,ind1]

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $40$ and $20$ for $argrelextrema$ and it gives quite reasonable and nice results
- [x] To give lower orders used if-else conditions on length of indices list

In [18]:
peak1_week=[]
peak2_week=[]
# week ID automatically becomes ''indices+1'' by adding one in indices

for i in range(len(district_ids_from_cases)):
    time_series=np.array(week_df[week_df['districtid']==district_ids_from_cases[i]].cases)
    indices=signal.argrelextrema(time_series,np.greater_equal, order=40)[0]
    if len(indices)==2:
        peak1_week.append(indices[0]+1)
        peak2_week.append(indices[1]+1)
    else:
        if len(indices)<2:
            indices=signal.argrelextrema(time_series,np.greater_equal, order=20)[0]
            if len(indices)==2:
                peak1_week.append(indices[0]+1)
                peak2_week.append(indices[1]+1)
            else:
                indices=choose_function_week(time_series,indices)
                peak1_week.append(indices[0]+1)
                peak2_week.append(indices[1]+1)
        else:
            indices=choose_function_week(time_series,indices)
            peak1_week.append(indices[0]+1)
            peak2_week.append(indices[1]+1)

# 585 cases pass len==2 condition at fiirst go

#### Unique peak values analysis(weeks)
- [x] there only seems 4-5 outliers in peak1 and peak2. Gives that $argrelextrema$ works just fine with appropriate order given to it.
- [x] It is quite possible that these outlier peaks are due to discrepencies/errors in recording data

In [19]:
print(np.unique(peak1_week,return_counts=True))
print(np.unique(peak2_week,return_counts=True))

(array([  1,  18,  20,  27,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  81,  87,  88, 121, 122]), array([ 1,  2,  1,  3,  1,  2,  3,  2,  3,  1,  3,  6, 10, 14, 14, 14, 29,
       10, 16,  8,  8, 20, 20, 21, 29, 43, 50, 47, 36, 20, 27, 17, 17,  9,
        7,  3, 11,  8,  9,  3,  6,  6,  5,  1,  4, 10, 16,  3,  1,  4,  9,
        1,  1,  1,  1,  1,  2,  2]))
(array([103, 107, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135,
       136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147]), array([ 1,  2,  2,  5,  6, 10, 24, 42, 51, 44, 76, 84, 73, 39, 35, 35, 27,
       13,  5,  4,  4,  2,  2,  1,  1,  2,  3,  1,  3,  4,  3,  5,  1,  2,
        5,  1,  4]))


#### Month timeline
- [x] I have used Active cases = # of active cases on last day of month(this is 14th of next month): as months are from 15th of previous month to 14th of next month


In [20]:
%%time
no_of_months=list(time_id_month.values())[-1]
districtid=[]
monthid=[]
cases=[]
for i in range(len(district_names_from_cases)): # if delhi etc are not there just 
    data_foo=data_csv[data_csv['District']==district_names_from_cases[i]]
    for j in range(no_of_months):
        districtid.append(district_ids_from_cases[i])
        monthid.append(j+1)
        foo_df=data_foo[data_foo['Month ID']==j+1]
        
        if foo_df.shape[0]==0:
            cases.append(0)
        else:
            cases.append(foo_df.Active.tolist()[-1])
month_df=pd.DataFrame({'districtid':districtid,'monthid':monthid,'cases':cases})

CPU times: user 12.1 s, sys: 0 ns, total: 12.1 s
Wall time: 12.1 s


#### Function that chooses between more than two peaks finded by $argrelextrema$
- [x] discription is given in python comments
- [x] there is one special case of MN_Churachandpur, where everything is zero, I have given it avg peak1 and peak2 values, manually.

In [21]:
def choose_function_month(time_series,indices):
    indices=indices.tolist()
     
    # find index of first max
    foo_list=time_series[indices]
    max1=max(foo_list.tolist())
    ind1=indices[foo_list.argmax()]
    indices.remove(ind1)
    time_series[ind1]=0
    
    #find index of second peak
    foo_list=time_series[indices]
    ind2=indices[foo_list.argmax()]
    
    ### for cases like MN_Churachandpur
    ### where there is no record of cases
    if max1==0:# there is only one district like this MN_Churachandpur 
      ind1=6  # if remove this then this gives peaks some cakcy value
      ind2=13 # for that giving a nice value around avg of other district peak
        
    # return index
    if ind1<ind2:
        return [ind1,ind2]
    else:
        return [ind2,ind1]

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $4$, $3$, and $2$ for $argrelextrema$ and it gives quite reasonable and nice results
- [x] To give lower orders used if-else conditions on length of indices list

In [22]:
peak1_month=[]
peak2_month=[]

# month ID automatically becomes ''indices+1'' by adding 1 in indices
for i in range(len(district_ids_from_cases)):
    time_series=np.array(month_df[month_df['districtid']==district_ids_from_cases[i]].cases)
    indices=signal.argrelextrema(time_series,np.greater_equal, order=4)[0]
    if len(indices)==2:
        peak1_month.append(indices[0]+1)
        peak2_month.append(indices[1]+1)
    else:
        if len(indices)<2:
            indices=signal.argrelextrema(time_series,np.greater_equal, order=3)[0]
            if len(indices)==2:
                peak1_month.append(indices[0]+1)
                peak2_month.append(indices[1]+1)
            else:
                if len(indices)<2:
                    indices=signal.argrelextrema(time_series,np.greater_equal, order=2)[0]
                    if len(indices)==2:
                        peak1_month.append(indices[0]+1)
                        peak2_month.append(indices[1]+1)
                    else:
                        indices=choose_function_month(time_series,indices)
                        peak1_month.append(indices[0]+1)
                        peak2_month.append(indices[1]+1)
                else:
                    indices=choose_function_month(time_series,indices)
                    peak1_month.append(indices[0]+1)
                    peak2_month.append(indices[1]+1)
        else:
            indices=choose_function_month(time_series,indices)
            peak1_month.append(indices[0]+1)
            peak2_month.append(indices[1]+1)


#### Unique peak values analysis(months)
- [x] there only seems 4-5 outliers in peak1 and peak2. Gives that $argrelextrema$ works just fine with appropriate order given to it.
- [x] It is quite possible that these outlier peaks are due to discrepencies/errors in recording data

In [23]:
print(np.unique(peak1_month,return_counts=True))
print(np.unique(peak2_month,return_counts=True))

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([  1,   1,   3,  20, 124, 295, 102,  42,  30,   4]))
(array([12, 13, 14, 15, 16, 17]), array([  1,  38, 506,  46,  21,  10]))


#### Gives districtid list for data frame

In [24]:
unique_district_ids_list=[]
for x in districtid:
    # check if exists in unique_list or not
    if x not in unique_district_ids_list:
        unique_district_ids_list.append(x)

#### converting overlapping weekid back to non-overlapping weekids
- [x] overlapping($1-148$) to non-overlapping($1-74$)
- [x] for odd overlapping week id add 1
- [x] now devide by 2 to get non-overlapping weekis

In [25]:
peak1=[]
peak2=[]
for i in range(len(peak1_week)):
    a=peak1_week[i]
    b=peak2_week[i]
    if a%2==0:
        peak1.append(int(a/2))
    else:
        peak1.append(int((a+1)/2))
    if b%2==0:
        peak2.append(int(b/2))
    else:
        peak2.append(int((b+1)/2))    

#### Write peaks(weeks and months) for districts

In [26]:
district_peaks_df=pd.DataFrame({'districtid':unique_district_ids_list,'wave1 − weekid':peak1, 'wave2 − weekid':peak2, 'wave1 − monthid':peak1_month, 'wave2 − monthid':peak2_month
})
district_peaks_df.sort_values("districtid",axis=0,ascending=True, inplace=True, kind='mergesort')
district_peaks_df.reset_index(inplace=True,drop=True)
district_peaks_df.to_csv('district-peaks.csv',index=False)

## Analysis for States

- [x] I have used district only from week_df/month_df to avoid extra counting of districts that are not there in the intersection. 

#### Add stateids to the week-df

In [27]:
state_list=[]
for i in range(week_df.shape[0]):
    state_list.append(week_df['districtid'][i].split('_')[0])
    
week_df['stateid']=state_list

#### Add stateids to the month-df

In [28]:
state_list=[]
for i in range(month_df.shape[0]):
    state_list.append(month_df['districtid'][i].split('_')[0])
    
month_df['stateid']=state_list

#### Get unique state IDs from week_df/month_df

In [29]:
stateid_unique=np.unique(np.array(week_df.stateid))

#### cases counting for states from week_df

In [30]:
# for weeks
stateid=[]
weeks=[]
case_count=[]
for i in range(len(stateid_unique)):
    data_foo=week_df[week_df['stateid']==stateid_unique[i]]
    for j in range(no_of_weeks):
        stateid.append(stateid_unique[i])
        weeks.append(j+1)
        foo_df=data_foo[data_foo['weekid']==j+1]
        case_count.append(sum(foo_df.cases.tolist()))
week_df=pd.DataFrame({'districtid':stateid,'weekid':weeks,'cases':case_count})

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $40$ and $20$ for $argrelextrema$ and it gives quite reasonable and nice results
- [x] To give lower orders used if-else conditions on length of indices list

In [31]:
peak1_week=[]
peak2_week=[]
# week ID automatically becomes ''indices+1'' by adding one in indices

for i in range(len(stateid_unique)):
    time_series=np.array(week_df[week_df['districtid']==stateid_unique[i]].cases)
    indices=signal.argrelextrema(time_series,np.greater_equal, order=30)[0]
    if len(indices)==2:
        peak1_week.append(indices[0]+1)
        peak2_week.append(indices[1]+1)
    else:
        if len(indices)<2:
            indices=signal.argrelextrema(time_series,np.greater_equal, order=25)[0]
            if len(indices)==2:
                peak1_week.append(indices[0]+1)
                peak2_week.append(indices[1]+1)
            else:
                indices=choose_function_week(time_series,indices)
                peak1_week.append(indices[0]+1)
                peak2_week.append(indices[1]+1)
        else:
            indices=choose_function_week(time_series,indices)
            peak1_week.append(indices[0]+1)
            peak2_week.append(indices[1]+1)

#### Unique peak values analysis(weeks)
- [x] there only seems 4-5 outliers in peak1 and peak2. Gives that $argrelextrema$ works just fine with appropriate order given to it.
- [x] It is quite possible that these outlier peaks are due to discrepencies/errors in recording data

In [32]:
print(np.unique(peak1_week,return_counts=True))
print(np.unique(peak2_week,return_counts=True))

(array([ 1, 38, 43, 48, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 63, 64, 73]), array([1, 1, 3, 1, 1, 3, 2, 6, 2, 2, 1, 1, 2, 1, 1, 1, 2]))
(array([114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 141,
       144]), array([1, 1, 1, 1, 1, 4, 5, 7, 1, 1, 4, 2, 1, 1]))


#### cases counting for states from month_df

In [33]:
# for months
stateid=[]
weeks=[]
case_count=[]
for i in range(len(stateid_unique)):
    data_foo=month_df[month_df['stateid']==stateid_unique[i]]
    for j in range(no_of_months):
        stateid.append(stateid_unique[i])
        weeks.append(j+1)
        foo_df=data_foo[data_foo['monthid']==j+1]
        case_count.append(sum(foo_df.cases.tolist()))
month_df=pd.DataFrame({'districtid':stateid,'monthid':weeks,'cases':case_count})

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $4$, $3$, and $2$ for $argrelextrema$ and it gives quite reasonable and nice results
- [x] To give lower orders used if-else conditions on length of indices list

In [34]:
peak1_month=[]
peak2_month=[]

# month ID automatically becomes ''indices+1'' by adding 1 in indices
for i in range(len(stateid_unique)):
    time_series=np.array(month_df[month_df['districtid']==stateid_unique[i]].cases)
    indices=signal.argrelextrema(time_series,np.greater_equal, order=4)[0]
    if len(indices)==2:
        peak1_month.append(indices[0]+1)
        peak2_month.append(indices[1]+1)
    else:
        if len(indices)<2:
            indices=signal.argrelextrema(time_series,np.greater_equal, order=3)[0]
            if len(indices)==2:
                peak1_month.append(indices[0]+1)
                peak2_month.append(indices[1]+1)
            else:
                if len(indices)<2:
                    indices=signal.argrelextrema(time_series,np.greater_equal, order=2)[0]
                    if len(indices)==2:
                        peak1_month.append(indices[0]+1)
                        peak2_month.append(indices[1]+1)
                    else:
                        indices=choose_function_month(time_series,indices)
                        peak1_month.append(indices[0]+1)
                        peak2_month.append(indices[1]+1)
                else:
                    indices=choose_function_month(time_series,indices)
                    peak1_month.append(indices[0]+1)
                    peak2_month.append(indices[1]+1)
        else:
            indices=choose_function_month(time_series,indices)
            peak1_month.append(indices[0]+1)
            peak2_month.append(indices[1]+1)

#### Unique peak values analysis(months)
- [x] there only seems 4-5 outliers in peak1 and peak2. Gives that $argrelextrema$ works just fine with appropriate order given to it.
- [x] It is quite possible that these outlier peaks are due to discrepencies/errors in recording data

In [35]:
print(np.unique(peak1_month,return_counts=True))
print(np.unique(peak2_month,return_counts=True))

(array([1, 5, 6, 7, 8, 9]), array([ 1,  4, 15,  9,  1,  1]))
(array([13, 14, 15, 16, 17]), array([ 2, 25,  2,  1,  1]))


#### converting overlapping weekid back to non-overlapping weekids
- [x] overlapping($1-148$) to non-overlapping($1-74$)
- [x] for odd overlapping week id add 1
- [x] now devide by 2 to get non-overlapping weekis

In [36]:
peak1=[]
peak2=[]
for i in range(len(peak1_week)):
    a=peak1_week[i]
    b=peak2_week[i]
    if a%2==0:
        peak1.append(int(a/2))
    else:
        peak1.append(int((a+1)/2))
    if b%2==0:
        peak2.append(int(b/2))
    else:
        peak2.append(int((b+1)/2))    

#### Write in csv

In [37]:
state_peaks_df=pd.DataFrame({'stateid':stateid_unique,'wave1 − weekid':peak1, 'wave2 − weekid':peak2, 'wave1 − monthid':peak1_month, 'wave2 − monthid':peak2_month
})
state_peaks_df.sort_values("stateid",axis=0,ascending=True, inplace=True, kind='mergesort')
state_peaks_df.reset_index(inplace=True,drop=True)
state_peaks_df.to_csv('state-peaks.csv',index=False)

## Analysis for Overall(India)

#### Get cases count for overall(weeks)

In [38]:
# for weeks
overallid='IN'
weeks=[]
case_count=[]
for j in range(no_of_weeks):
    stateid.append(overallid)
    weeks.append(j+1)
    case_count.append(sum(week_df[week_df.weekid==j+1].cases.tolist()))
week_df=pd.DataFrame({'districtid':overallid,'weekid':weeks,'cases':case_count})

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $40$ for $argrelextrema$ and it gives quite reasonable and nice results

In [39]:
peak1_week=[]
peak2_week=[]

time_series=np.array(week_df[week_df['districtid']==overallid].cases)
indices=signal.argrelextrema(time_series,np.greater_equal, order=40)[0]
if len(indices)==2:
    peak1_week.append(indices[0]+1)
    peak2_week.append(indices[1]+1)
else:
    indices=choose_function_week(time_series,indices)
    peak1_week.append(indices[0]+1)
    peak2_week.append(indices[1]+1)

#### converting overlapping weekid back to non-overlapping weekids
- [x] overlapping($1-148$) to non-overlapping($1-74$)
- [x] for odd overlapping week id add 1
- [x] now devide by 2 to get non-overlapping weekis

In [40]:
peak1=[]
peak2=[]
for i in range(len(peak1_week)):
    a=peak1_week[i]
    b=peak2_week[i]
    if a%2==0:
        peak1.append(int(a/2))
    else:
        peak1.append(int((a+1)/2))
    if b%2==0:
        peak2.append(int(b/2))
    else:
        peak2.append(int((b+1)/2))    

#### Get cases count for overall(months)

In [41]:
# for months
overallid='IN'
weeks=[]
case_count=[]
for j in range(no_of_months):
    stateid.append(overallid)
    weeks.append(j+1)
    case_count.append(sum(month_df[month_df.monthid==j+1].cases.tolist()))
month_df=pd.DataFrame({'districtid':overallid,'monthid':weeks,'cases':case_count})

#### Find peaks with $argrelextrema$
- [x] I have used(trial and error) order $4$ for $argrelextrema$ and it gives quite reasonable and nice results

In [42]:
peak1_month=[]
peak2_month=[]

time_series=np.array(month_df[month_df['districtid']==overallid].cases)
indices=signal.argrelextrema(time_series,np.greater_equal, order=4)[0]
if len(indices)==2:
    peak1_month.append(indices[0]+1)
    peak2_month.append(indices[1]+1)
else:
    indices=choose_function_month(time_series,indices)
    peak1_month.append(indices[0]+1)
    peak2_month.append(indices[1]+1)

#### Write in overall csv

In [43]:
overall_peaks_df=pd.DataFrame({'overallid':overallid,'wave1 − weekid':peak1, 'wave2 − weekid':peak2, 'wave1 − monthid':peak1_month, 'wave2 − monthid':peak2_month
})
overall_peaks_df.to_csv('overall-peaks.csv',index=False)